In [28]:
import numpy as np
import pandas as pd

In [29]:
def find_previous_digits(line_list, current_index):
    i = current_index - 1
    while i >= 0:
        previous_digits = line_list[i][:9]
        if previous_digits and previous_digits[0].isdigit():
            return previous_digits
        i -= 1

def calculate_average(value):
    if '-' in value:
        start, end = map(int, value.split('-'))
        return (start + end) // 2
    else:
        return int(value)

def process_data(data_set):
    result = []
    for i in range(len(data_set)):
        first_element = data_set[i][0]
        if first_element.isdigit():
            result.append([calculate_average(cell) for cell in data_set[i][:9]])
        else:
            previous_digits = find_previous_digits(data_set, i)
            if previous_digits:
                result.append([calculate_average(cell) for cell in previous_digits])
    return result

def yarn_fun(size_total_needed_m, name, company, m_per_skein, price_per_skein):
    skeins_needed_before_ceil = size_total_needed_m / m_per_skein
    skeins_needed = np.ceil(skeins_needed_before_ceil)
    total_price = skeins_needed * price_per_skein
    return name, company, skeins_needed_before_ceil, skeins_needed, total_price

In [30]:
with open('monday_sweater.txt', 'r') as file:
    original_data = file.read().splitlines()

processed_data = [line.replace('Materialer: ', '').replace('(', '').replace(')', '') for line in original_data]
processed_data = processed_data[0].split('strikkes sammen med')
ds1 = processed_data[0].split('eller')
ds2 = processed_data[1].split('eller')
ds1 = [line.split(' ') for line in ds1]
ds2 = [line.split(' ') for line in ds2]
ds1 = [list(filter(None, line)) for line in ds1]
ds2 = [list(filter(None, line)) for line in ds2]

second_last = np.array([int(line[-2]) for dataset in [ds1, ds2] for line in dataset])
fifth_last = np.array([int(line[-5]) for dataset in [ds1, ds2] for line in dataset])
m_g_ratio = second_last/fifth_last
ds1_m_g_ratio = m_g_ratio[:len(ds1)]
ds2_m_g_ratio = m_g_ratio[len(ds1):]

result_ds1 = process_data(ds1)*ds1_m_g_ratio[:, np.newaxis]
result_ds2 = process_data(ds2)*ds2_m_g_ratio[:, np.newaxis]

XS1, S1, M1, L1, XL1, XXL1, XXXL1, XXXXL1, XXXXXL1 = np.mean(result_ds1, axis=0)
XS2, S2, M2, L2, XL2, XXL2, XXXL2, XXXXL2, XXXXXL2 = np.mean(result_ds2, axis=0)

In [45]:
chosen_size = ['M','L','XL']

"""Number are for the monday sweater"""
size_total_needed_m1 = [XS1, S1, M1, L1, XL1, XXL1, XXXL1, XXXXL1, XXXXXL1]
size_total_needed_m2 = [XS2, S2, M2, L2, XL2, XXL2, XXXL2, XXXXL2, XXXXXL2]

In [47]:
sizes = ['XS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL', 'XXXXL', 'XXXXXL']

Arwetta_Filcolana = ['Arwetta', 'Filcolana', 50, 210, 46]
Alva_Filcolana = ['Alva', 'Filcolana', 25, 175, 30]
Alpakka1_Isager = ['Alpakka1', 'Isager', 50, 400, 70]
Sunday_Sandnes = ['Sunday', 'Sandnes Garn', 50, 235, 57]
FriendsExtraFineMerino_Hobbii = ['Friends Extra Fine Merino', 'Hobbii', 50, 180, 49]
EasyCareMerino_Mayflower = ['Easy Care Merino', 'Mayflower', 50, 185, 40]
Anina_Filcolana = ['Anina', 'Filcolana', 50, 210, 42]
BabyMerino_Drops = ['Baby Merino', 'Drops', 50, 175, 23]
MerinoExtraFine_Drops = ['Merino Extra Fine', 'Drops', 50, 105, 22]
FriendsWool_Hobbii = ['Friends Wool', 'Hobbii', 50, 100, 19]
SoftAlpacaLace_Hobbii = ['Soft Alpaca Lace', 'Hobbii', 50, 400, 80]
SoftAlpaca_Hobbii = ['Soft Alpaca', 'Hobbii', 50, 165, 56]
Alpaca_Drops = ['Alpaca', 'Drops', 50, 167, 22]
Rustica_Smuksak = ['Rustica', 'Smuksak', 100, 350, 80]


primary = [Arwetta_Filcolana, Sunday_Sandnes, FriendsExtraFineMerino_Hobbii, EasyCareMerino_Mayflower, Anina_Filcolana, BabyMerino_Drops, MerinoExtraFine_Drops, FriendsWool_Hobbii, SoftAlpaca_Hobbii, Alpaca_Drops, Rustica_Smuksak]
secondary = [Arwetta_Filcolana, Alva_Filcolana, Alpakka1_Isager, SoftAlpacaLace_Hobbii, Alpaca_Drops]
df = pd.DataFrame({'size': sizes})
chosen_size_indices = df[df['size'].isin(chosen_size)].index.tolist()


In [48]:
for k in chosen_size_indices:
    list_of_results = []
    for i in primary:
        for j in secondary:
            name1, company1, skein_before_ceil1, skein1, price1 = yarn_fun(size_total_needed_m1[k], i[0], i[1], i[3], i[4])
            name2, company2, skein_before_ceil2 ,skein2, price2 = yarn_fun(size_total_needed_m2[k], j[0], j[1], j[3], j[4])
            list_of_results.append([name1, name2, company1, company2, skein1, skein2, 
                                    ((skein1-skein_before_ceil1)*100).round(0), ((skein2-skein_before_ceil2)*100).round(0), 
                                    ((skein1-skein_before_ceil1)*i[3]).round(-1), ((skein2-skein_before_ceil2)*j[3]).round(-1), 
                                    price1 + price2])
    table = pd.DataFrame(list_of_results, columns = ['p', 's', 'company_p', 'company_s', 'n_p', 'n_s', 'perc_p', 'perc_s', 'm_extr_p', 'm_extr_s', 'price'])
    table = table.sort_values('price')
    table = table.reset_index(drop=True)
    name = sizes[k]
    title = f'Size {name}: Total needed primary: {size_total_needed_m1[k]:.0f} m. Total needed secondary: {size_total_needed_m2[k]:.0f} m'
    print(title)
    print(table)
    print('  ')

Size M: Total needed primary: 1205 m. Total needed secondary: 1283 m
                            p                 s     company_p  company_s  \
0                 Baby Merino            Alpaca         Drops      Drops   
1                      Alpaca            Alpaca         Drops      Drops   
2                 Baby Merino              Alva         Drops  Filcolana   
3                      Alpaca              Alva         Drops  Filcolana   
4                Friends Wool            Alpaca        Hobbii      Drops   
5                       Anina            Alpaca     Filcolana      Drops   
6           Merino Extra Fine            Alpaca         Drops      Drops   
7                 Baby Merino          Alpakka1         Drops     Isager   
8                     Arwetta            Alpaca     Filcolana      Drops   
9                      Alpaca          Alpakka1         Drops     Isager   
10           Easy Care Merino            Alpaca     Mayflower      Drops   
11                B